In [1]:
%matplotlib inline

In [3]:
import data_processor

In [4]:
data = data_processor.load_data("data2016.mat")

In [6]:
data_processor.create_features(data, "brushing")

[[-0.44360425]
 [-0.49416625]
 [-0.53943559]
 ...
 [-0.28608298]
 [-0.27807829]
 [ 0.0773344 ]]
